# Ejemplos de _networking_ con Python

**Recuerda que al leer este documento directamente desde GitHub, no estás aprovechando su contenido al 100%.
Editando el código puedes aprender más que leyéndolo. Te recomendamos descargarlo y ejecutarlo desde tu computador para que puedas hacer modificaciones e interactuar con el código.**

**IMPORTANTE: El siguiente material incluye ejemplos que deberían ser ejecutados en programas separados, donde uno actúa como el cliente, y otro actúa como el servidor. Para ello, copie los códigos correspondientes en archivos .py en su computador y ejecútelo en consolas separadas. Sí ejecuta los códigos directamente en este jupyter-notebook los programas se pueden quedar pegados esperando una conexión. "**


En este *notebook*, veremos ejemplos de conexión y transmisión de archivos con Python.

# Tabla de contenidos

1. [Envío de JSON](#Envío-de-JSON)
    1. [Envío de datos con `pickle`](#Envío-de-datos-con-pickle)
    2. [Envío de muchos datos](#Envío-de-muchos-datos)
    3. [Ejemplo completo: servidor con manejo de múltiples clientes en forma concurrente](#Ejemplo-completo:-servidor-con-manejo-de-múltiples-clientes-en-forma-concurrente)

## Envío de JSON

En el siguiente ejemplo, veremos cómo programar un servidor que reciba datos y los envíe de vuelta al cliente, lo que se conoce como un *echo server*.

**Intenta hacer la prueba con dos computadores: en uno ejecuta el código de la siguiente celda con la definición del servidor, y en el otro la celda subsiguiente con la definición del cliente que envía los datos JSON**.

In [ ]:
# Implementación del servidor que recibe datos y los envía de vuelta.
# Esto comúnmente se denomina como 'echo server'.
import socket

host = socket.gethostname()   # Esto obtiene el nombre en la red de este host
port = 12345

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
print("Listo para recibir. Ahora voy a esperar que me hablen...")
sock.listen()

sock_cliente, (host_cliente, puerto_cliente) = sock.accept()
print("Conexión desde", host_cliente, puerto_cliente)

while True:
    data = sock_cliente.recv(4096)
    print(f"Recibí estos bytes: {data}")
    if not data:
        break
    print("Los enviaré de vuelta")
    sock_cliente.sendall(data)

sock_cliente.close()
sock.close()

Listo para recibir. Ahora voy a esperar que me hablen...


El cliente envía datos JSON y los imprime una vez que el servidor se los envía de vuelta. Luego cierra la conexión.


In [ ]:
# Implementación del cliente que envía los datos en formato JSON.
# Pon atención en la serialización y transformación a bytes.

import json
import socket

server_host = ''  # Debemos poner aquí la dirección IP del servidor.
                  # Si no ponemos nada, supone que estamos hablando con un programa en el mismo host.
server_port = 12345

# Generamos la información que enviaremos; en este caso, es un simple diccionario.
data = {1: "Hola", 2: "Chao"}
mensaje = json.dumps(data)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_host, server_port))

# Debemos enviar bytes.
sock.sendall(mensaje.encode('utf-8'))

# Decodificamos los bytes y luego los deserializamos con JSON.
data = json.loads(sock.recv(4096).decode('utf-8'))
print(data)
input("ENTER para terminar.")
sock.close()

Notemoa que cuando el cliente termina, la conexión son el servidor se cierra. Esto se traduce en que el servidor dejar de recibir datos y, de acuerdo a cómo está construido, también termina. Si queremos que el servidor quede disponible para recibir otras conexiones, debemos volver a ejecutar `listen`.

## Envío de datos con `pickle`

Al igual que el ejemplo anterior, podemos enviar cualquier objeto de Python serializado con `pickle`. El siguiente código muestra un ejemplo de cómo conectarse al servidor anterior y enviarle datos serializados con `pickle`. Cuando los *bytes* vienen de vuelta desde el servidor lo deserializamos y tenemos nuevamente la instancia de la clase `Persona` que habíamos enviado.

Puedes usar este cliente con **el mismo servidor (*echo server*) del ejemplo anterior**. El *echo server* no se preocupa de qué tipo de datos recibe. Sólo recibe bytes y los manda de vuelta.

In [ ]:
import pickle
import socket

server_host = ''  # Debemos poner aquí la dirección IP del servidor.
                  # Si no ponemos nada, supone que estamos hablando con un programa en el mismo host.
port = 12345


class Persona:
    def __init__(self, nombre, correo):
        self.nombre = nombre
        self.correo = correo

# Enviaremos esta instancia de la clase Persona.
persona = Persona("Juan Pérez", "jp@ejemplo.com")
mensaje = pickle.dumps(persona)

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_host, port))
sock.sendall(mensaje)

data = pickle.loads(sock.recv(4096))
print(f"He recibido bytes de vuelta y los deserialicé a {type(data)}")
print(data.nombre)
sock.close()

Además, podríamos modificar el código del servidor para que haga alguna acción requerida con los datos que recibe y envíe el resultado de esta acción de vuelta. Por ejemplo, podríamos verificar en el servidor los datos de un cliente para iniciar sesión. 

**Recomendamos fuertemente, dentro de lo posible, conectar dos computadores y probar enviando hacia y desde ambas partes**, de tal forma de familiarizarse lo más posible con los *sockets* y disfrutar de los beneficios que nos ofrece la red. Para ello, debes ingresar en la variable *host*, la **dirección IP del servidor** en el caso de los clientes. Los servidores no necesitan modificación.

**¿Cómo conseguir la dirección IP de un computador?.** El método varía dependiendo de cada sistema operativo, pero usualmente hay interfaces de configuración de la red en las cuales se puede averiguar ese dato. Nada que una búsqueda específica en Internet para tu sistema operativo no pueda resolver. Atención que si preguntas en sitios como [https://whatismyipaddress.com/](whatismyipaddress.com) encontrarás la dirección pública que te asigna tu proveedor de Internet, pero si quieres conectarte con un computador que está en tu misma casa (i.e. en tu misma subred) seguramente la dirección IP será distinta pues se trata de una dirección "interna" a tu subred.

## Envío de muchos datos

Supongamos que queremos transmitir un archivo relativamente grande entre dos computadores. Lo único que tenemos que hacer es leer el archivo en *bytes*, y transmitirlo con un *socket*. Implementemos el código del emisor y del receptor del archivo.

In [ ]:
# Este es el receptor del archivo.
import socket

host = socket.gethostname()
port = 12345

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen()
print("Escuchando...")

# Aceptamos un cliente.
sock_cliente, (host_cliente, port_cliente) = sock.accept()
print("Conexión entrante aceptada.")

# Leemos la información y la guardamos en un archivo.
datos = sock_cliente.recv(4096)
with open('files/recibido.bin', 'wb') as binfile:
    binfile.write(datos)

print("¡Archivo recibido!")
# Le enviamos una respuesta a la contraparte.
sock_cliente.sendall("Gracias.".encode('utf-8'))

# Cerramos los sockets.
sock_cliente.close()
sock.close()

In [ ]:
# Este es el emisor del archivo.
import socket

host_receptor = socket.gethostname()   ## Si lo ejecutas en otro computador, aquí debes poner 
                                       ## debes poner su dirección IP
puerto_receptor = 12345

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Nos conectamos al receptor del archivo, que ya debería estar escuchando.
sock.connect((host_receptor, puerto_receptor))
print("Conexión establecida.")

# Leemos el archivo y lo enviamos.
with open('files/enviar.bin', 'rb') as binfile:
    datos = binfile.read()
    sock.sendall(datos)

print("¡Archivo enviado!")

# Imprimimos lo que nos responda la contraparte.
print("Respuesta:", sock.recv(4096).decode('utf-8'))

# Cerramos el socket.
sock.close()

Veamos lo que sucede al tener cada celda en un programa distinto.

![](imgs/send-large-file.png)

Todo parece ir bien pero, si comparamos los archivos, vemos diferencias significativas de tamaño.

![](imgs/compare-large-files.png)

¡Leímos muy pocos *bytes*! Podríamos cambiar el valor en `recv(buffer)`, pero si hacemos esto seguimos teniendo problemas:

- ¿Cómo saber de antemano el tamaño de lo que voy a recibir?
- E incluso sabiendo el tamaño del mensaje que voy a recibir, ¿qué ocurre si tengo que recibir *megabytes* o *gigabytes* de datos?

La solución a este problema tiene dos partes. Primero, tener un **protocolo** o esquema de comunicación para nuestra aplicación de transferencia de archivos. Por ahora, sólo necesitamos una forma de indicar el tamaño del archivo que queremos mandar. Una forma simple es enviar un número que ocupe cuatro *bytes* para indicar el tamaño del archivo, antes de enviar el archivo en sí. Si usamos 4 *bytes* como encabezado de todos los mensajes para indicar el largo de estos, podremos enviar mensajes de hasta $2^{32}$ *bytes* (4 GiB). Esta es una cantidad razonable para la mayoría de los casos.

Ya sabiendo el número de *bytes* que tenemos que leer, tenemos que encargarnos de leer el archivo por *chunks*, hasta completarlo. Tomando eso en cuenta, volvamos a implementar nuestro emisor y receptor de archivos.

In [ ]:
# Receptor mejorado del archivo.
import socket

host = socket.gethostname()
puerto = 12345

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, puerto))
sock.listen()
print('Escuchando...')

# Aceptamos un cliente.
sock_cliente, (host_cliente, puerto_cliente) = sock.accept()
print("Conexión entrante aceptada.")

# Leemos los 4 bytes del tamaño del archivo.
# Con esto transformamos una serie de bytes en un int.
largo_archivo = int.from_bytes(sock_cliente.recv(4), byteorder='big')
datos = bytearray()
bytes_leidos = 0
print(f"OK. Ahora sé que debe recibido {largo_archivo} bytes")

# Ahora leemos el archivo por chunks, de máximo 4096 bytes.
while len(datos) < largo_archivo:
    # El último recv será probablemente más chico que 4096
    bytes_leer = min(4096, largo_archivo - len(datos))
    datos_recibidos = sock_cliente.recv(bytes_leer)
    # Recordemos que el método recv, entrega una cantidad máxima, pero no nos asegura que nos 
    # entregue los 4096 bytes. Es por esto, que la cantidad de bytes que hemos recibido en
    # total, se deben ver siempre en función de lo que retornó el método recv, y no lo que
    # le entregamos como parámetro
    bytes_leidos += len(datos_recibidos)
    print(f"He recibido {len(datos_recibidos)} bytes en el último recv. Van {bytes_leidos} en total.")
    datos.extend(datos_recibidos)

print(f"¡Listo! He recibido {len(datos)} bytes")

# Guardamos la información en un archivo.
with open('files/recibido.bin', 'wb') as binfile:
    binfile.write(datos)

print("¡Archivo guardado!")
# Le enviamos una respuesta a la contraparte.
sock_cliente.sendall("Gracias.".encode('utf-8'))

# Cerramos los sockets.
sock_cliente.close()
sock.close()

In [ ]:
# Emisor mejorado del archivo.
import socket

host_receptor = socket.gethostname()
puerto_receptor = 12345

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Nos conectamos al receptor del archivo, que ya debería estar escuchando.
sock.connect((host_receptor, puerto_receptor))
print("Conexión establecida.")

# Leemos el archivo y lo enviamos.
with open('files/enviar.bin', 'rb') as binfile:
    datos = binfile.read()
    largo_archivo = len(datos)
    print(f"Voy a enviar {largo_archivo} bytes")
    # 1. Enviar el tamaño del archivo/
    # (int.to_bytes transforma un entero en una cantidad de bytes por el primer parámetro)
    sock.sendall(largo_archivo.to_bytes(4, byteorder='big'))
    # 2. Enviar el archivo.
    sock.sendall(datos)

print("¡Archivo enviado!")

# Imprimirmos lo que nos responda la contraparte.
print("Respuesta:", sock.recv(4096).decode('utf-8'))

# Cerramos el socket.
sock.close()

Ahora podemos ver que funciona correctamente.

![](imgs/send-large-file-improved.png)
![](imgs/compare-large-files-improved.png)

## Ejemplo completo: servidor con manejo de múltiples clientes en forma concurrente

En la práctica, lo más probable es que tengamos que manejar múltiples usuarios de forma concurrente en el servidor, y además enviar paquetes de datos de tamaños arbitrarios. A continuación, describiremos cómo incorporaremos estas modificaciones.

El manejo de varios usuarios simultáneamente se logra mediante el uso de *threads*. Cada vez que se ejecutan los métodos `socket.accept()` y `socket.recv()` los *threads* donde ellos son invocados se bloquean hasta aceptar una nueva conexión (en el caso del primero), y recibir datos (en el caso del segundo). Para evitar este comportamiento tendremos que crear un *thread* encargado de: 

- Aceptar nuevos clientes; 
- Y, cada vez que se acepte un cliente, crear un *thread* nuevo que se ocupe de escuchar y enviar información al nuevo cliente conectado.

Además, implementaremos lo aprendido sobre cómo enviar o recibir mensajes de tamaño arbitrario. Definamos a continuación una clase `Client` donde ejemplificaremos las modificaciones mencionadas.

In [ ]:
import socket
import threading


class Client:
    """
    Maneja toda la comunicación desde el lado del cliente.

    Implementa el esquema de comunicación donde los primeros 4 bytes de cada
    mensaje indicarán el largo del mensaje enviado.
    """

    def __init__(self, port, host):
        print("Inicializando cliente...")

        self.host = host
        self.port = port
        self.socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        try:
            self.connect_to_server()
            self.listen()
            self.repl()
        except ConnectionError:
            print("Conexión terminada.")
            self.socket_client.close()
            exit()

    def connect_to_server(self):
        """Crea la conexión al servidor."""

        self.socket_client.connect((self.host, self.port))
        print("Cliente conectado exitosamente al servidor.")

    def listen(self):
        """
        Inicializa el thread que escuchará los mensajes del servidor.

        Es útil hacer un thread diferente para escuchar al servidor,
        ya que de esa forma podremos tener comunicación asíncrona con este.
        Luego, el servidor nos podrá enviar mensajes sin necesidad de
        iniciar una solicitud desde el lado del cliente.
        """

        thread = threading.Thread(target=self.listen_thread, daemon=True)
        thread.start()

    def send(self, msg):
        """
        Envía mensajes al servidor.

        Implementa el mismo protocolo de comunicación que mencionamos;
        es decir, agregar 4 bytes al principio de cada mensaje
        indicando el largo del mensaje enviado.
        """

        msg_bytes = msg.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        self.socket_client.sendall(msg_length + msg_bytes)

    def listen_thread(self):
        while True:
            response_bytes_length = self.socket_client.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            # Recibimos datos hasta que alcancemos la totalidad de los datos
            # indicados en los primeros 4 bytes recibidos.
            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(self.socket_client.recv(read_length))

            print(f"{response.decode()}\n>>> ", end='')

    def repl(self):
        """
        Captura el input del usuario.

        Lee mensajes desde el terminal y después se pasan a `self.send()`.
        """

        print("------ Consola ------\n>>> ", end='')
        while True:
            msg = input()
            response = self.send(msg)


if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    client = Client(port, host)

Ahora implemetaremos la clase `Server` que se encargará de controlar toda la lógica en el lado del servidor:

In [ ]:
import socket
import threading


class Server:
    def __init__(self, port, host):
        print("Inicializando servidor...")

        self.host = host
        self.port = port
        self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.bind_and_listen()
        self.accept_connections()

    def bind_and_listen(self):
        """
        Enlaza el socket creado con el host y puerto indicado.

        Primero, se enlaza el socket y luego queda esperando
        por conexiones entrantes.
        """
        self.socket_server.bind((self.host, self.port))
        self.socket_server.listen()
        print(f"Servidor escuchando en {self.host}:{self.port}...")

    def accept_connections(self):
        """
        Inicia el thread que aceptará clientes.

        Aunque podríamos aceptar clientes en el thread principal de la
        instancia, es útil hacerlo en un thread aparte. Esto nos
        permitirá realizar la lógica en la parte del servidor sin dejar
        de aceptar clientes. Por ejemplo, seguir procesando archivos.
        """
        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()

    def accept_connections_thread(self):
        """
        Es arrancado como thread para aceptar clientes.

        Cada vez que aceptamos un nuevo cliente, iniciamos un
        thread nuevo encargado de manejar el socket para ese cliente.
        """
        print("Servidor aceptando conexiones...")

        while True:
            client_socket, _ = self.socket_server.accept()
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket, ),
                daemon=True)
            listening_client_thread.start()

    @staticmethod
    def send(value, sock):
        """
        Envía mensajes hacia algún socket cliente.

        Debemos implementar en este método el protocolo de comunicación
        donde los primeros 4 bytes indicarán el largo del mensaje.
        """
        stringified_value = str(value)
        msg_bytes = stringified_value.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        sock.send(msg_length + msg_bytes)

    def listen_client_thread(self, client_socket):
        """
        Es ejecutado como thread que escuchará a un cliente en particular.

        Implementa las funcionalidades del protocolo de comunicación
        que permiten recuperar la informacion enviada.
        """
        print("Servidor conectado a un nuevo cliente...")

        while True:
            response_bytes_length = client_socket.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(client_socket.recv(read_length))

            received = response.decode()

            if received != "":
                # El método `self.handle_command()` debe ser definido.
                # Este realizará toda la lógica asociado a los mensajes
                # que llegan al servidor desde un cliente en particular.
                # Se espera que retorne la respuesta que el servidor
                # debe enviar hacia el cliente.
                response = self.handle_command(received, client_socket)
                self.send(response, client_socket)

    def handle_command(self, received, client_socket):
        print("Comando recibido:", received)
        # Este método debería ejecutar la acción y enviar una respuesta.
        return "Acción asociada a " + received

if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    server = Server(port, host)

Las clases descritas en los códigos anteriores definen entidades que podrán ser usadas eficientemente a través de *threads* para comunicación asíncrona. Es recomendable mantener las funcionalidades de estas clases sólo destinadas a funciones de *networking*, i.e., solo para efectuar la comunicación entre el servidor y los clientes. Será natural que intentemos agregar código que permita manejar la lógica en el cliente o el servidor de acuerdo a los requerimientos de nuestro problema, lo que tendrá como consecuencia la generación de código muy **difícil de leer y mantener**. 

Por lo tanto, recomendamos fuertemente efectuar la implementación de la lógica en otras clases (*backend*).

<font size='1' face='Arial'><sup>1</sup>Agradecemos al ex-ayudante del curso, Rodolfo Palma, por su colaboración para la elaboración del ejemplo práctico de este material.</font>